# Wyniki dla 30 rejonow i kilku POI dla Warszawy - listopad 2018

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format

## read data
* od matrices: Prt, PuT

In [12]:
od_PuT = pd.read_csv('data//MTX_99321.csv')
del od_PuT['Unnamed: 0']
od_PuT.columns = [int(c) for c in od_PuT.columns]
od_PrT = pd.read_csv('data//MTX_99421.csv')
del od_PrT['Unnamed: 0']
od_PrT.columns = [int(c) for c in od_PrT.columns]

In [13]:
od_PrT = od_PrT.stack().reset_index()
od_PrT.columns=['Z_Rejon',"Do_Rejon", 'TripsPrT']
od_PuT = od_PuT.stack().reset_index()
od_PuT.columns=['Z_Rejon',"Do_Rejon", 'TripsPuT']
od = pd.merge(od_PrT, od_PuT, on = ['Z_Rejon', "Do_Rejon"])

* trips

In [14]:
trips = pd.read_csv('data//From_Via_To.csv')
del trips['Unnamed: 0']

## prepare

In [15]:
trips.Czas_PrT = trips.Czas_PrT/60 # przeliczam na minuty
trips.Czas_PuT = trips.Czas_PuT/60 *0.5 # tu zakladam dwa razy krotsze, dla obliczen, w Warszawie tak nie bedzie
BUDGET_PRT = trips['Czas_PrT'].quantile(0.5) # przyjmuje ze liczy sie tylko dolna polowa czasow
BUDGET_PUT = BUDGET_PRT #trips['Czas_PuT'].quantile(0.5) # a tu juz zakladam ze budzety sa rowne
trips['time_left_PrT']=BUDGET_PRT - trips.Czas_PrT # ile czasu zostaje ponizej budzetu w POI
trips['time_left_PuT']=BUDGET_PUT - trips.Czas_PuT
trips.Z_Rejon = trips.Z_Rejon.astype(int)
trips.Do_Rejon = trips.Do_Rejon.astype(int)
pd.options.display.float_format = '{:,.1f}'.format

In [16]:
trips = pd.merge(trips,od, on = ['Z_Rejon',"Do_Rejon"])
trips['odPair'] = trips["Z_Rejon"].astype(int)*1000+trips["Do_Rejon"].astype(int)

# OD pairs-based analysis:
## number of POI available
- total minutes
- mean/median minutes

In [19]:
print('number of POIs within budget per OD Pair - PuT/PrT')
nPOIs = trips[trips.Czas_PrT<BUDGET_PRT].groupby(by=['Z_Rejon',"Do_Rejon"])['POI'].nunique().to_frame('nPOIsPRT')
nPOIs['nPOIsPUT'] = trips[trips.Czas_PuT<BUDGET_PUT].groupby(by=['Z_Rejon',"Do_Rejon"])['POI'].nunique()
nPOIs.reset_index().head()

number of POIs within budget per OD Pair - PuT/PrT


,Z_Rejon,Do_Rejon,nPOIsPRT,nPOIsPUT
0,1,1,5,4
1,1,2,5,4
2,1,3,4,4
3,1,4,5,4
4,1,5,5,3


## mean/median time available

In [21]:
print("time per OD pair  PrT")
t = trips[trips.Czas_PrT<BUDGET_PRT].groupby(
    by=['Z_Rejon',"Do_Rejon"])['Czas_PrT', 'time_left_PrT'].agg(['sum','mean','median']).reset_index()
t.head()

time per OD pair  PrT


Z_Rejon Do_Rejon Czas_PrT             time_left_PrT            
                        sum mean median           sum mean median
0       1        1     59.4 11.9    8.0          93.3 18.7   22.5
1       1        2     56.1 11.2    8.0          96.5 19.3   22.5
2       1        3     28.0  7.0    6.2          94.1 23.5   24.3
3       1        4     58.3 11.7    8.0          94.3 18.9   22.5
4       1        5     59.2 11.8    8.0          93.4 18.7   22.5

In [23]:
print("time per OD pair PuT")
t = trips[trips.Czas_PuT<BUDGET_PUT].groupby(
    by=['Z_Rejon',"Do_Rejon"])['Czas_PuT', 'time_left_PuT'].agg(['sum','mean','median']).reset_index()
t.head()

time per OD pair PuT


Z_Rejon Do_Rejon Czas_PuT             time_left_PuT            
                        sum mean median           sum mean median
0       1        1     91.1 22.8   23.8          31.1  7.8    6.8
1       1        2     86.5 21.6   23.2          35.6  8.9    7.4
2       1        3     89.8 22.4   22.3          32.3  8.1    8.2
3       1        4     93.0 23.3   23.7          29.1  7.3    6.8
4       1        5     66.9 22.3   22.3          24.7  8.2    8.2